# Understanding HybridAGI prompting

In this notebook we are going to show you how HybridAGI build the prompts for the LLM, that way you can understand how exactly the Agent use tools. To do so, we are going to make the simplest agent possible and inspect the LLM calls.

In [1]:
import dspy
import hybridagi.core.graph_program as gp
from hybridagi.memory.integration.local import LocalProgramMemory
from hybridagi.core.datatypes import AgentState, Query
from hybridagi.modules.agents import GraphInterpreterAgent
from hybridagi.modules.agents.tools import SpeakTool, PredictTool

main = gp.GraphProgram(
	name = "main",
	description = "The main program",
)

main.add(gp.Action(
    id = "answer",
	tool = "Speak",
	purpose = "Answer the user question",
	prompt = "Please answer to the Objective's question",
))

main.connect("start", "answer")
main.connect("answer", "end")

# We build the program to verify its structure
main.build()

program_memory = LocalProgramMemory(index_name="prompting")

program_memory.update(main)

agent_state = AgentState()

tools = [
    SpeakTool(
        agent_state = agent_state
    ),
    PredictTool(),
]

agent = GraphInterpreterAgent(
    program_memory = program_memory,
    agent_state = agent_state,
    tools = tools,
    verbose = False,
)

# We can now setup the LLM using Ollama client from DSPy

lm = dspy.OllamaLocal(model='mistral', max_tokens=1024, stop=["\n\n\n"])
dspy.configure(lm=lm)

# And call our agent

result = agent(Query(query="What is the city of the spatial and robotics industry in France?"))

print(lm.history[-1]["prompt"])



/home/yoan/.cache/pypoetry/virtualenvs/hybridagi-B1GoJrSC-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Given the fields `objective`, `context`, `purpose`, `prompt`, produce the fields `message`.

---

Follow the following format.

Objective: The long-term objective (what you are doing)

Context: The previous actions (what you have done)

Purpose: The purpose of the action (what you have to do now)

Prompt: The action specific instructions (How to do it)

Message: The message to send to the user

---

Objective: What is the city of the spatial and robotics industry in France?

Context:
--- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spatial and robotics industry in France?
--- END OF TRACE ---

Purpose: Answer the user question

Prompt: Please answer to the Objective's question

Message:


In [2]:
print(result.final_answer)

The city of the spatial and robotics industry in France is Toulouse. This is because Toulouse hosts several major aerospace companies such as Airbus, Thales Alenia Space, and the European Space Agency (ESA). Additionally, it has a strong robotics community with institutions like the Institute for Research in Computer Science, Robotics, and Automation of Grenoble (INRIA) and the National Institute of Applied Sciences (INSA) Toulouse.


In [3]:
# This is the last (and final) answer of the Agent but we can inspect the resulting trace like that
print("\n".join(result.program_trace.steps))

--- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spatial and robotics industry in France?
--- Step 1 ---
Action Purpose: Answer the user question
Action: {
  "message": "The city of the spatial and robotics industry in France is Toulouse. This is because Toulouse hosts several major aerospace companies such as Airbus, Thales Alenia Space, and the European Space Agency (ESA). Additionally, it has a strong robotics community with institutions like the Institute for Research in Computer Science, Robotics, and Automation of Grenoble (INRIA) and the National Institute of Applied Sciences (INSA) Toulouse."
}


#### The Agent Context

The context always contains the N (the param `num_history` of the agent) last steps of the program trace. At each step, the long-term objective of the program and the short-term purpose of the step to help the system behave, note that the Action's prompt is not present in the resulting step of the trace, this decision have been made to isolate the prompts and give the possibility to use multiple "roles" LLM in the same program seamlessly.

In [4]:
# Now a program that involve two actions to better understand the trace objective

main = gp.GraphProgram(
	name = "main",
	description = "The main program",
)

main.add(gp.Action(
    id = "elaborate",
	tool = "Predict",
	purpose = "Elaborate on the Objective question",
	prompt = "Please elaborate on the Objective's question",
))

main.add(gp.Action(
    id = "answer",
	tool = "Speak",
	purpose = "Answer the user question",
	prompt = "Please answer to the Objective's question",
))

main.connect("start", "elaborate")
main.connect("elaborate", "answer")
main.connect("answer", "end")

# Verify its structure
main.build()

# Update the program in memory
program_memory.update(main)

result = agent(Query(query="What is the city of the spatial and robotics industry in France?"))

print(lm.history[-1]["prompt"])

Given the fields `objective`, `context`, `purpose`, `prompt`, produce the fields `message`.

---

Follow the following format.

Objective: The long-term objective (what you are doing)

Context: The previous actions (what you have done)

Purpose: The purpose of the action (what you have to do now)

Prompt: The action specific instructions (How to do it)

Message: The message to send to the user

---

Objective: What is the city of the spatial and robotics industry in France?

Context:
--- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spatial and robotics industry in France?
--- Step 1 ---
Action Purpose: Elaborate on the Objective question
Action: {
  "answer": "The city with a significant presence in the spatial (aerospace) and robotics industry in France is Toulouse. This city is often referred to as \"La Ville Rose\" or \"The Pink City.\" It is home to several major aerospace manufacturers, including Airbus, ATR, and the European Space Agency's (ESA) largest 

In [5]:
print(result.final_answer)

The city of France with a significant presence in both the spatial (aerospace) and robotics industry is Toulouse, often referred to as "La Ville Rose" or "The Pink City." It houses major aerospace manufacturers like Airbus, ATR, and the European Space Agency's largest establishment outside its headquarters in the Netherlands. Additionally, it has a robust robotics industry with numerous research centers and companies focusing on robotics and automation.


In [6]:
print("\n".join(result.program_trace.steps))

--- Step 0 ---
Call Program: main
Program Purpose: What is the city of the spatial and robotics industry in France?
--- Step 1 ---
Action Purpose: Elaborate on the Objective question
Action: {
  "answer": "The city with a significant presence in the spatial (aerospace) and robotics industry in France is Toulouse. This city is often referred to as \"La Ville Rose\" or \"The Pink City.\" It is home to several major aerospace manufacturers, including Airbus, ATR, and the European Space Agency's (ESA) largest establishment outside of its headquarters in the Netherlands. The robotics industry also has a strong presence in Toulouse, with numerous research centers and companies focusing on robotics and automation."
}
--- Step 2 ---
Action Purpose: Answer the user question
Action: {
  "message": "The city of France with a significant presence in both the spatial (aerospace) and robotics industry is Toulouse, often referred to as \"La Ville Rose\" or \"The Pink City.\" It houses major aerospace